レース結果のスクレイピング

import

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import os
import re
import datetime

レース結果をスクレイピング

In [3]:
#レースIDリスト生成
#開催年(2022と2023)
years  = [str(i).zfill(4) for i in range(2022, 2024)]
#開催場所(MAX10) 01札幌、02函館、03福島、04新潟、05東京、
#                06中山、07中京、08京都、09阪神、10小倉
place = [str(i).zfill(2) for i in range(1, 11)]
#開催回(MAX8)
times = [str(i).zfill(2) for i in range(1, 9)]
#開催日(MAX8)
days  = [str(i).zfill(2) for i in range(1, 9)]
#レース12(MAX12)
races = [str(i).zfill(2) for i in range(1, 13)]

raceIdList = []
for y in years:
    for p in place:
        for t in times:
            for d in days:
                for r in races:
                    raceIdList.append(y + p + t + d + r)

#URL
pfx = 'https://db.netkeiba.com/race/'
#readIdのリスト作成

colName = ['raceId','htmlBytes']    #列名をリストで生成
df = pd.DataFrame(columns=colName)  #データフレームの作成
escapeList = []
#pickleファイルの存在チェック
if(os.path.isfile('./data/race_html.pkl')):
    #pickleファイルを読み込む
    df = pd.read_pickle('./data/race_html.pkl')
    #読み込んだデータフレームから除外Idリストを作成
    escapeList = df['raceId'].to_list()

#除外リスト生成関数
def addEscapeList(id :str, ll :list):
    #raceIdを分解してlist化
    idAry = [id[0:4], id[4:6], id[6:8], id[8:10],id[10:12]]
    for r in range(1, 13):
        idAry[4] = str(r).zfill(2)
        ll.append(''.join(idAry))
    if idAry[3] == '01':
        for d in range(2,9):
            idAry[3] = str(d).zfill(2)
            #ll = addEscapeList(''.join(idAry), ll)
            ll.extend(addEscapeList(''.join(idAry), []))
    if idAry[2] == '01':
        for t in range(2,9):
            idAry[2] = str(t).zfill(2)
            #ll = addEscapeList(''.join(idAry), ll)
            ll.extend(addEscapeList(''.join(idAry), []))
    return ll

#raceIdリスト分ループ
for raceId in tqdm(raceIdList):
    try:
        if raceId in escapeList:
            continue
        url = pfx + raceId
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser')
        # 結果が存在しているページかチェック
        if 'レース結果' in soup.text:
            #データフレームを生成して一時変数に保存
            tmpDf = pd.DataFrame([[raceId, html.content]],columns=colName)
            #データフレームを結合
            df = pd.concat([df, tmpDf],axis=0,ignore_index=True)
        else:
            #不要なページだったら除外リストを更新
            escapeList = addEscapeList(raceId, escapeList)

        time.sleep(1) #マナー間隔をあける
    except:
        print('exception catch')
        break
df.to_pickle('./data/race_html.pkl') #データフレームの保存

100%|██████████| 15360/15360 [02:59<00:00, 85.52it/s] 


In [ ]:
df

HTMLからデータを生成

In [24]:
#保存したhtmlデータを読込
htmlDf = pd.read_pickle('./data/race_html.pkl')
#空のデータフレームを生成
raceResultDf = pd.DataFrame()

#保存したhtmlを1つずつ処理
for idx, dat in tqdm(htmlDf.iterrows(), total = len(htmlDf)):
    #idとバイナリーデータを取り出す
    raceId = dat['raceId']
    htmlBytes = dat['htmlBytes']

    soup = BeautifulSoup(htmlBytes, 'html.parser')
    table = soup.find_all('table')[0]

    #ヘッダー用のList生成してデータフレームを生成
    columns = []
    #thタグをひとつずつ取り出してListに追加
    for head in table.find_all('th'):
        columns.append(head.text)
    #作ったヘッダーListにraceId,horseId,jockeyId,trainerId列を追加
    columns = ['raceId'] + columns + ['horseId', 'jockeyId', 'trainerId']
    df = pd.DataFrame(columns = columns)

    #テーブルを1行毎に処理
    for i, row in enumerate(table.find_all('tr')):
        #最初の行はヘッダー列なので処理をスキップ
        if i == 0:
            continue
        items = [raceId]    #最初のデータにraceId
        #1行内のtdをすべて取り出す
        cells = row.find_all('td')
        #1データずつ改行コードを削除素ながらデータに追加
        for cell in cells:
            items.append(cell.text.replace('\n', ''))
        #リンク先を解析しながらhorseId,jockeyId,trainerIdを切り取ってデータに追加
        items.append(str(cells[3]).split('/horse/')[1].split('/')[0])
        items.append(str(cells[6]).split('/recent/')[1].split('/')[0])
        items.append(str(cells[18]).split('/recent/')[1].split('/')[0])
        #1個分のデータを追加
        df.loc[i] = items
#最後に1レース分のデータフレームを追加
raceResultDf = pd.concat([raceResultDf, df], axis=0)


100%|██████████| 5575/5575 [04:29<00:00, 20.66it/s]


In [25]:
items

['202308020512',
 '14',
 '8',
 '14',
 'クリノホノオ',
 '牡6',
 '53',
 '田口貫太',
 '1:55.0',
 'ハナ',
 '',
 '7-8-12-13',
 '40.1',
 '101.4',
 '12',
 '486(0)',
 '',
 '',
 '',
 '[西]谷潔',
 '栗本博晴',
 '',
 '2017101036',
 '01208',
 '00431']

In [ ]:
raceResultDf

前処理

In [29]:
df = raceResultDf.copy()    #加工用にコピー
#着外や除外などのデータを欠損データに変換
df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
df.dropna(subset=['着順'], inplace=True)
#性齢を性と年齢に分別
df['性'] = df['性齢'].map(lambda x: str(x)[0])
df['年齢'] = df['性齢'].map(lambda x: str(x)[1:])
#馬体重と体重増減を分割して計測不のデータは欠損データに変換
df['馬体重'] = df['馬体重'].map(lambda x: '---(-)' if '不' in x else x)
df['体重増減'] = df['馬体重'].str.split('(', expand=True)[1].str[:-1]
df['馬体重'] = df['馬体重'].str.split('(', expand=True)[0]
df['馬体重'] = pd.to_numeric(df['馬体重'], errors='coerce')
df.dropna(subset = ['馬体重'], inplace=True)
df['体重増減'] = pd.to_numeric(df['体重増減'], errors='coerce')
df.dropna(subset = ['体重増減'], inplace=True)
#調教師から拠点列を取り出す
df['拠点'] = df['調教師'].map(lambda x:\
    '東' if '[東]' in x else \
        '西' if '[西]' in x else \
            '地' if '[地]' in x else '外')
# ,を除去後、空の行は0で埋める
df['賞金'] = df['賞金(万円)'].str.replace(',','')
df['賞金'] = pd.to_numeric(df['賞金'], errors='coerce')
df['賞金'] = df['賞金'].fillna(0)

#文字列から数値に変換
for clna in ['着順','年齢','馬体重','体重増減','枠番','馬番']:
    df[clna] = df[clna].astype(int)
for clna in ['単勝','人気','賞金','斤量']:
    df[clna] = df[clna].astype(float)

#タイムを秒に変換
df['タイム'] = df['タイム'].map(lambda x: '10:0' if x == '' else x)
df['タイム'] = df['タイム'].map(lambda \
    x: float(x.split(':')[0]) * 60 + float(x.split(':')[1]))
df['タイム'] = df['タイム'].map(lambda x: np.nan if x > 550 else x)
#着差は1着タイムからの差分を計算して埋める
df['着差'] = df.groupby('raceId')['タイム'].transform(lambda x: x - x.min())

#一応出馬表と同じように馬番で行をソート
df = df.sort_values(['raceId','馬番']).reset_index(drop=True)

#必要な列だけ拾って列を並び替え
columns = ['raceId','枠番','馬番','horseId','馬名','性','年齢',
           '斤量','騎手','jockeyId','単勝','人気','調教師',
           'trainerId','拠点','馬体重','体重増減','着順',
           'タイム','着差','通過','上り','賞金']
df = df[columns].copy()

df.to_pickle('./data/race_result.pkl')

In [ ]:
df

レース情報

In [31]:
#保存したhtmlデータを読み込む
htmlDf = pd.read_pickle('./data/race_html.pkl')

#空のリストの生成
raceInfoList = []

# 保存したhtmlを1つずつ処理
for idx, dat in tqdm(htmlDf.iterrows(), total = len(htmlDf)):
    #idとバイナリーデータを取り出す
    raceId = dat['raceId']
    htmlBytes = dat['htmlBytes']

    #BeautifulSoupでバイナリーデータを解析
    soup = BeautifulSoup(htmlBytes.decode('euc-jp', 'ignore'), 'html.parser')
    
    #情報部分を指定して取り出す
    mainrace_data = soup.find('div', class_='mainrace_data')

    #1レース分のデータを辞書型で宣言
    rowdata = {}
    #raceId
    rowdata['raceId'] = raceId
    #レース名を抽出
    rowdata['レース名'] = mainrace_data.find('h1').text
    #レースNoを抽出
    rowdata['R'] = mainrace_data.find('dt').text\
        .replace('\n', '').replace(' ', '').replace('R', '')
    
    #レース情報部のテキストを取得して'/'で分割
    spantexts = mainrace_data.find('span').text\
        .replace('\xa0', '').replace(' ', '').split('/')
    #特定の文字列があるかどうかコース種を抽出
    rowdata['コース種'] = '障害' if '障' in spantexts[0] else \
                        'ダート' if 'ダ' in spantexts[0] else '芝'
    #レースの回りを抽出 ※開催場所で固定
    rowdata['コース回り'] = '右' if '右' in spantexts[0] else \
                            '左' if '左' in spantexts[0] else '障害'
    #距離を抽出
    rowdata['距離'] = int(re.findall('\d+', spantexts[0])[0])
    #天気、馬場状態を抽出
    rowdata['天気'] = spantexts[1][3:]
    rowdata['馬場'] = spantexts[2].split(':')[1]
    #発送時間を抽出 ※必要かは不明
    rowdata['発送'] = spantexts[3][3:]

    #次のレース情報部を取得していらない部分を削除して' 'で分割
    smalltxt = mainrace_data.find('p',class_='smalltxt').text\
        .replace('\xa0', ' ').replace('  ', ' ').split(' ')
    #開催日をタイムスタンプに変換し、フォーマットを指定して保存
    dt = datetime.datetime.strptime(smalltxt[0], '%Y年%m月%d日')
    rowdata['日付'] = dt.strftime('%Y/%m/%d')
    #開催場所
    placeDict = {'01':'札幌', '02':'函館', '03':'福島', '04':'新潟', '05':'東京',
                 '06':'中山', '07':'中京', '08':'京都', '09':'阪神', '10':'小倉'}
    rowdata['開催場所'] = placeDict[raceId[4:6]]
    #レースのグレードを判別
    if 'G1' in rowdata['レース名']:
        raceGrade = 'G1'
    elif 'G2' in rowdata['レース名']:
        raceGrade = 'G2'
    elif 'G3' in rowdata['レース名']:
        raceGrade = 'G3'
    elif '未勝利' in smalltxt[2]:
        raceGrade = '未勝利'
    elif '新馬' in smalltxt[2]:
        raceGrade = '新馬'
    elif '1勝' in smalltxt[2] or '500万' in smalltxt[2]:
        raceGrade = '1勝クラス'
    elif '2勝' in smalltxt[2] or '1000万' in smalltxt[2]:
        raceGrade = '2勝クラス'
    elif '3勝' in smalltxt[2] or '1600万' in smalltxt[2]:
        raceGrade = '3勝クラス'
    else:
        raceGrade = 'オープン'
    rowdata['グレード'] = raceGrade
    #出走制限を特定の文字列で判定
    if '牡・牝' in smalltxt[3]:
        restriction = '牡・牝'
    elif '牝' in smalltxt[3]:
        restriction = '牝'
    else:
        restriction = '無'
    rowdata['制限'] = restriction
    #重量制限を特定の文字列で判定
    if 'ハンデ' in smalltxt[3]:
        handicap = 'ハンデ'
    elif '別定' in smalltxt[3]:
        handicap = '別定'
    else:
        handicap = '定量'
    rowdata['ハンデ'] = handicap
    #1レース分のデータをリストに追加
    raceInfoList.append(rowdata)
#辞書型をデータフレームに変換
raceInfoDf = pd.DataFrame(raceInfoList)
#データフレームをpickleデータに保存
raceInfoDf.to_pickle('./data/race_info.pkl')
 

100%|██████████| 5575/5575 [07:04<00:00, 13.14it/s]


In [ ]:
raceInfoDf

血統データをスクレイピング

In [33]:
#レース結果読み込み
raceResultDf = pd.read_pickle('./data/race_result.pkl')
#URL
pfx = 'https://db.netkeiba.com/horse/ped/'

colName = ['horseId','htmlBytes']    #列名をリストで生成
df = pd.DataFrame(columns=colName)  #データフレームの作成
escapeList = []
#pickleファイルの存在チェック
if(os.path.isfile('./data/ped_html.pkl')):
    #pickleファイルを読み込む
    df = pd.read_pickle('./data/ped_html.pkl')
    #読み込んだデータフレームから除外Idリストを作成
    escapeList = df['horseId'].to_list()

#raceIdリスト分ループ
for horseId in tqdm(raceResultDf['horseId'].unique()):
    try:
        if horseId in escapeList:
            continue
        url = pfx + horseId
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser')
        table = soup.find_all('table')[0]
        # 結果が存在しているページかチェック
        if len(table.find_all('a')) != 0:
            #データフレームを生成して一時変数に保存
            tmpDf = pd.DataFrame([[horseId, html.content]],columns=colName)
            #データフレームを結合
            df = pd.concat([df, tmpDf], axis=0, ignore_index=True)
        
        time.sleep(1) #マナー間隔をあける
    except:
        print('exception catch')
        break
    
df.to_pickle('./data/ped_html.pkl') #データフレームの保存

100%|██████████| 14/14 [00:00<?, ?it/s]


血統データ

In [34]:
#保存したhtmlデータを読み込む
htmlDf = pd.read_pickle('./data/ped_html.pkl')
#念のためにインディックスを振り直しておく
htmlDf = htmlDf.reset_index(drop=True)

#血統を取り出す
tergetList = [i for i in range(62)]

#列名を生成
columns = ['horseId']
for i in range(62):
    columns.append('pedName_' + str(i))
    columns.append('pedId_' + str(i))
#空のデータフレームを生成
horsePedDf = pd.DataFrame(columns=columns)

for idx, dat in tqdm(htmlDf.iterrows(), total = len(htmlDf)):
    #horseidとhtmlBytesを取り出す
    horseId = dat['horseId']
    htmlBytes = dat['htmlBytes']

    #BeautifulSoupでHTMLを解析
    soup = BeautifulSoup(htmlBytes.decode('euc-jp','ignore'), 'html.parser')
    tds = soup.find_all('table')[0].find_all('td')

    #最初はhorseId
    rowdata = [horseId]
    for lno in tergetList:
        #名前部分の抽出
        rowdata.append(tds[lno].text.split('\n')[1])
        #IDの抽出
        rowdata.append(str(tds[lno]).split('/horse/')[1].split('/')[0])
    #rowdataをデータフレームに追加
    horsePedDf.loc[idx] = rowdata

#データフレーム保存
horsePedDf.to_pickle('./data/horse_ped.pkl')

100%|██████████| 26/26 [00:01<00:00, 15.48it/s]


競走馬毎レースのスクレイピング

In [35]:
raceResultDf = pd.read_pickle('./data/race_result.pkl')
#競走馬IDリストの生成
horseIdList = raceResultDf['horseId'].unique().tolist()

#スクレイピング対象か買うににレース情報を読み込む
raceInfoDf = pd.read_pickle('./data/race_info.pkl')
#日付を確認したいからテーブル結合
raceResultDfM = pd.merge(raceResultDf, raceInfoDf,
                        on='raceId', how='left', suffixes=['', '_right'])

#url
pfx = 'https://db.netkeiba.com/horse/result/'

colName = ['horseId', 'htmlBytes']
df = pd.DataFrame(columns=colName)
#pickleファイルチェック
if (os.path.isfile('./data/horse_html.pkl')):
    df = pd.read_pickle('./data/horse_html.pkl')
#エラー対策をしながら空のデータフレームを生成
horseResultDf = pd.DataFrame(columns=['horseId','raceId','日付'])
if(os.path.isfile('./data/horse_result_pkl')):
   horseResultDf = pd.read_pickle('./data/horse_result.pkl')

for horseId in tqdm(horseIdList):
    if horseId in horseResultDf['horseId'].unique().tolist():
        chkR = raceResultDfM[raceResultDfM['horseId']==horseId].\
                    sort_values('日付').tail(1)['日付'].iloc[0]
        chkH = horseResultDf[horseResultDf['horseId']==horseId].\
                    sort_values('日付').tail(1)['日付'].iloc[0]
        if chkR != chkH:
            #直近のレースの日付が一致しなかったらデータを消して再度スクレイピング
            df = df[df['horseId'] != horseId]
        else:
            #一致したらスキップ
            continue
    try:
        url = pfx + horseId
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser')
        #結果が存在しているページかチェック
        if len(soup.find_all('table')) != 0:
            tmpDf = pd.DataFrame([[horseId,html.content]], columns=colName)
            #結合
            df = pd.concat([df, tmpDf], axis=0, ignore_index=True)
        
        time.sleep(1)
    except:
        print('exception catch')
        break
df.to_pickle('./data/horse_html.pkl')

100%|██████████| 14/14 [00:18<00:00,  1.35s/it]


In [ ]:
horseIdList

htmlからデータを抽出(戦績データ)

In [27]:
#保存したhtmlデータを読み込む
htmlDf = pd.read_pickle('./data/horse_html.pkl')
#空のデータフレームを生成
horseResultDf = pd.DataFrame()

#保存したhtmlを1つずつ処理
for idx, dat in tqdm(htmlDf.iterrows(), total=len(htmlDf)):
    horseId = dat['horseId']
    htmlBytes = dat['htmlBytes']

    if horseId in escapeList:
        continue

    #BeautifulSoupでHTMLを解析
    soup = BeautifulSoup(htmlBytes.decode('euc-jp', 'ignore'), 'html.parser')
    table = soup.find_all('table')[0]

    #ヘッダー用のList生成してデータフレームを生成
    columns = ['horseId']
    #thタグをひとつずつ取り出してListに追加
    for head in table.find_all('th'):
        columns.append(head.text)
    columns += ['raceId', 'jockeyId']
    #空のデータフレームを生成
    df = pd.DataFrame(columns=columns)
    #cellごとに分解してlistに追加
    for i, row in enumerate(table.find_all('tr')):
        #最初の行はヘッダー列なので処理をスキップ
        if i == 0:
            continue
        #競走馬IDを埋め込み
        items = [horseId]
        cells = row.find_all('td')
        #cellごとに分解してlistに追加
        for cell in cells:
            items.append(cell.text.replace('\n', ''))
        #readIdとjockeyを抽出してlistに追加
        items.append(str(cells[4]).split('/race/')[1].split('/')[0])
        try:
            items.append(str(cells[12]).split('/racent/')[1].split('/')[0])
        except:
            #たまに騎手にアンカーがいない人がいるから例外処理を追加
            items.append('xxxxx')
        #データフレームに追加
        df.loc[i] = items
    #データフレームに結合
    horseResultDf = pd.concat([horseResultDf, df], axis=0)
#なんか重複するときがあるからhorseIdと日付をキーに重複行削除
horseResultDf = horseResultDf[~horseResultDf[['horseId', '日付']].duplicated()]

100%|██████████| 134/134 [00:09<00:00, 14.51it/s]


前処理

In [28]:
df = horseResultDf.copy()
#着外や除外などのデータを欠損データに変換
df['着順'] = pd.to_numeric(df['着順'],errors='coerce')
df.dropna(subset=['着順'], inplace=True)
#馬体重と体重増減を分割して計測不のデータは欠損データに変換
df['馬体重'] = df['馬体重'].map(lambda x: '---(-)' if '不' in x else x)
df['体重増減'] = df['馬体重'].str.split('(', expand=True)[1].str[:-1]
df['馬体重'] = df['馬体重'].str.split('(', expand=True)[0]
df['馬体重'] = pd.to_numeric(df['馬体重'], errors='coerce')
df.dropna(subset=['馬体重'], inplace=True)
#,を除去後、空の行は0で埋める
df['賞金'] = df['賞金'].str.replace(',','')
df['賞金'] = pd.to_numeric(df['賞金'], errors='coerce')
df['賞金'] = df['賞金'].fillna(0)
#レース結果と書式を合わせるために、オッズは単勝に列名を変える
df['単勝'] = df['オッズ']
#変換不可能なデータを欠損データに変換
df['上り'] = pd.to_numeric(df['上り'], errors='coerce')
df.dropna(subset={'上り'}, inplace=True)

#文字列から数値に変換
for clna in ['R', '頭数', '着順', '馬体重', '体重増減', '枠番' ,'馬番']:
    df[clna] = df[clna].astype(int)
for clna in ['単勝', '人気', '賞金', '斤量', '上り', '着差']:
    df[clna] = df[clna].astype(float)

#タイムは扱いやすく為に秒に変換しておく
df['タイム'] = df['タイム'].map(lambda x: '10:0' if x == '' else x)
df['タイム'] = df['タイム'].map(lambda \
                          x:float(x.split(':')[0]) * 60 + float(x.split(':')[1]))
df['タイム'] = df['タイム'].map(lambda x: np.nan if x > 550 else x)

#horseIdと日付でソート
df = df.sort_values(['horseId','日付'], ascending=[True, False]).reset_index(drop=True)
for horseId in tqdm(df['horseId'].unique()):
    x = df[df['horseId']==horseId]['日付'].to_list()
    ilist = []
    for i in range(len(x) - 1):
        interval = datetime.datetime.strptime(x[i], '%Y/%m/%d') - \
                    datetime.datetime.strptime(x[i+1], '%Y/%m/%d')
        ilist.append(interval.days)
    ilist += [0]
    df.loc[df['horseId']==horseId, '出走間隔'] = ilist
df['出走間隔'] = df['出走間隔'].astype(int)

#必要な列だけ拾って、列を並べ替え
columns = ['horseId', '日付', 'R', 'レース名', 'raceId', '頭数',
           '枠番', '馬番', '単勝', '人気', '着順', '騎手', 'jockeyId',
           '斤量', 'タイム', '着差', '通過', 'ペース', '上り', '馬体重',
           '体重増減', '出走間隔', '賞金']
df.to_pickle('./data/horse_result.pkl')

100%|██████████| 26/26 [00:00<00:00, 1366.60it/s]
